In [1]:
import pandas as pd
import numpy as np
import toad
import warnings
warnings.filterwarnings("ignore")
from utils.blocks import build_train_test,build_metrics

In [73]:
# build data
np.random.seed(20211201)
d1 = np.arange(100).tolist()
np.random.shuffle(d1)
d1[88] = np.nan
d3 = np.random.choice(list('abcdefghijklmnopqrst'), size=100).tolist()
d3[67:75] = [np.nan]*len(d3[67:75])
data_dict = {
    'col1': d1,
    'col2': [ round(i, 0) for i in np.random.uniform(18, 55, size=100).tolist()],
    'col3': d3,
    'col4': np.random.choice(['sc', 'cq', 'gz', 'sx','ah','fj','gz','bj','sh','tj','hb','jx','gx','hn'], size=100).tolist(),
    'target': np.random.choice([0,1], size=100).tolist()
}

# data = pd.DataFrame(data_dict).to_numpy()
# x_data, y_data = data[:, :4], data[:, 4]
data = pd.DataFrame(data_dict)
x_data, y_data = data.iloc[:, :4], data.iloc[:, 4]
train_test = build_train_test(data)
train_index, test_index = next(train_test)
x_train, y_train = x_data.iloc[train_index.tolist(), :], y_data[train_index.tolist()]
x_test, y_test = x_data.iloc[test_index.tolist(), :], y_data[test_index.tolist()]

In [77]:
train_data = pd.concat([x_train, y_train], axis=1)
c = toad.transform.Combiner()
c.fit(train_data, y='target', method='dt')

transer = toad.transform.WOETransformer()

train_woe = transer.fit_transform(c.transform(train_data), train_data['target'], exclude=['target'])
print(train_woe.head())

        col1      col2      col3      col4  target
92  0.014599 -0.208545  0.014599 -0.390866       1
28 -1.084013 -0.390866  0.995428  0.397591       0
37 -1.712622 -0.614010  0.014599 -1.307157       0
83  0.079137  0.707746 -0.653231 -1.307157       0
3  -1.712622 -0.208545  0.995428 -0.796331       0


In [75]:
# from toad.plot import bin_plot

# col = 'col1'
# bin_plot(c.transform(data[[col,'target']], labels=True), x=col, target='target')

In [80]:
card = toad.ScoreCard(
    combiner = c,
    transer = transer,
    #class_weight = 'balanced',
    #C=0.1,
    #base_score = 600,
    #base_odds = 35 ,
    #pdo = 60,
    #rate = 2
)

card.fit(train_woe.iloc[:, :4], train_woe['target'])

ScoreCard(combiner=<toad.transform.Combiner object at 0x7f8f5faecd10>,
          transer=<toad.transform.WOETransformer object at 0x7f8f5d258890>)

In [79]:
card

ScoreCard(combiner=<toad.transform.Combiner object at 0x7f8f5faecd10>,
          transer=<toad.transform.WOETransformer object at 0x7f8f5d258890>)

In [81]:
import numpy as np

np.unique([0,0,0,1,1,0,1,1,1,1,1,0], return_counts=True)

(array([0, 1]), array([5, 7]))